# <center> **502 Project** </center>
### <center> Team member: Chenxi Liu, Nuo Tian, Mengyu Yu, Yuan Liu </center>

### Data Selection: 
#### Yahoo News Data Set (Part 1 and 2of 35) 




### STEP 1 : Data Extracting and Processing 

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder \
    .appName("project")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=project>

#### 1. Build Schema Using the README Instruction of the Data Set 
#### 

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("article_type", StringType(), True),
    StructField("np1", StringType(), True),
    StructField("np2", StringType(), True),
    StructField("context", StringType(), True),
    StructField("source", StringType(), True),
    StructField("category", StringType(), True),
    StructField("location", StringType(), True),
    StructField("time", StringType(), True),])

#### 2. Readin the Data Set 
#### 

In [3]:
df = spark.read.csv("s3://anly502project/data/part-r-00000",sep = "\t",header=False,schema=schema)
df_2 = spark.read.csv("s3://anly502project/data/part-r-00001",sep = "\t",header=False,schema=schema)

#### 3. Verify the Schema 
#### 

In [4]:
#### Data Schema
df.printSchema()
df_2.printSchema()

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)

root
 |-- article_type: string (nullable = true)
 |-- np1: string (nullable = true)
 |-- np2: string (nullable = true)
 |-- context: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- time: string (nullable = true)



#### 4. Combine the Data Set
#### 

In [5]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [6]:
unioned_df = unionAll([df, df_2])


In [7]:
#### show combined
unioned_df.show(10)

+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|article_type|            np1|np2|             context|       source|            category|            location| time|
+------------+---------------+---+--------------------+-------------+--------------------+--------------------+-----+
|     article|    Dark Knight|  E|  arg1 and Wall arg2|             |intlnews topstor ...|      , kerala india|14299|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|     article|    Communities|  E|arg1 mobilised in...|             |    politics topstor|                   ,|14026|
|     article|    Carotenoids|  E|arg1 and caroteno...|             |topstor,health,sc...|                   ,|14660|
|            |     Coast bias|  E|arg2 is for East ...|             |      sports topstor| columbus, ohio u...|13956|
|     article|Commerce office|  E|arg1 at DDD Linco...| 

In [8]:
#### show number of the rows 
unioned_df.count()

196017242

In [9]:
####Drop unrelated Column
unioned_df = unioned_df.drop("source").drop("article_type").drop("location").drop("time")

In [10]:
####Show data frame after dropping unrelated columns 
unioned_df.show(10)

+---------------+---+--------------------+--------------------+
|            np1|np2|             context|            category|
+---------------+---+--------------------+--------------------+
|    Dark Knight|  E|  arg1 and Wall arg2|intlnews topstor ...|
|    Carotenoids|  E|arg1 and caroteno...|topstor,health,sc...|
|    Communities|  E|arg1 mobilised in...|    politics topstor|
|    Carotenoids|  E|arg1 and caroteno...|topstor,health,sc...|
|     Coast bias|  E|arg2 is for East ...|      sports topstor|
|Commerce office|  E|arg1 at DDD Linco...|topstor,lifestle,...|
| 75-minute mark|  E|arg1 in a pulsati...|      topstor,sports|
| Brigham Circle|  E|      arg2 past arg1|             topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews headlin...|
| Brigham Circle|  E|arg2 line service...|             topstor|
+---------------+---+--------------------+--------------------+
only showing top 10 rows



#### 5. Select the very top category by using multiple split 
#### 

In [11]:
from pyspark.sql.functions import split
split_col = split(unioned_df['category'], ',')
unioned_df = unioned_df.withColumn('category', split_col.getItem(0))

In [12]:
unioned_df.show(10)

+---------------+---+--------------------+--------------------+
|            np1|np2|             context|            category|
+---------------+---+--------------------+--------------------+
|    Dark Knight|  E|  arg1 and Wall arg2|intlnews topstor ...|
|    Carotenoids|  E|arg1 and caroteno...|             topstor|
|    Communities|  E|arg1 mobilised in...|    politics topstor|
|    Carotenoids|  E|arg1 and caroteno...|             topstor|
|     Coast bias|  E|arg2 is for East ...|      sports topstor|
|Commerce office|  E|arg1 at DDD Linco...|             topstor|
| 75-minute mark|  E|arg1 in a pulsati...|             topstor|
| Brigham Circle|  E|      arg2 past arg1|             topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews headlin...|
| Brigham Circle|  E|arg2 line service...|             topstor|
+---------------+---+--------------------+--------------------+
only showing top 10 rows



In [13]:
split_col_2 = split(unioned_df['category'], ' ')
unioned_df = unioned_df.withColumn('category', split_col_2.getItem(0))

In [14]:
split_col_3 = split(unioned_df['category'], '_')
unioned_df = unioned_df.withColumn('category', split_col_3.getItem(0))

In [15]:
split_col_4 = split(unioned_df['category'], '-')
unioned_df = unioned_df.withColumn('category', split_col_4.getItem(0))

In [16]:
#### Show data frame after filtering the category
unioned_df.show(10)

+---------------+---+--------------------+---------+
|            np1|np2|             context| category|
+---------------+---+--------------------+---------+
|    Dark Knight|  E|  arg1 and Wall arg2| intlnews|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|    Communities|  E|arg1 mobilised in...| politics|
|    Carotenoids|  E|arg1 and caroteno...|  topstor|
|     Coast bias|  E|arg2 is for East ...|   sports|
|Commerce office|  E|arg1 at DDD Linco...|  topstor|
| 75-minute mark|  E|arg1 in a pulsati...|  topstor|
| Brigham Circle|  E|      arg2 past arg1|  topstor|
| Brigham Circle|  E| arg2 trains at arg1|localnews|
| Brigham Circle|  E|arg2 line service...|  topstor|
+---------------+---+--------------------+---------+
only showing top 10 rows



#### 6. Category (label) insight 
#### 

In [17]:
#### Get some insight of the label 
unioned_df.groupBy("category").count().show()

+------------+-------+
|    category|  count|
+------------+-------+
|      travel|    890|
|      ciridi|     70|
|      cirmus|     76|
|      cirngc|      3|
|      cirrfs|      2|
|regionalnews|1280737|
|      launch|   3652|
|      cirhge|   2520|
|      cirmec|    171|
|      cje2de|    223|
|      cje2ds|     51|
|      cirsvu|      3|
|      cirhw6|   3173|
|       cirgw|    367|
|      cirgha|    279|
|      cje3fq|     54|
|      cje2fa|     43|
|      cirlwu|      2|
|      cirida|  14761|
|      cirfvc|   1809|
+------------+-------+
only showing top 20 rows



In [18]:
#### Filter out the null and empty category 
unioned_df = unioned_df.filter(unioned_df.category.isNotNull())
unioned_df = unioned_df.filter(unioned_df.category != '')

In [19]:
#### More insight 
from pyspark.sql.functions import countDistinct
unioned_df.agg(countDistinct(unioned_df.category).alias('distinct_label')).collect()
##### We can see we have around 1116 distinct labels which some of it does not make any sense so we dicided to filter it out 

KeyboardInterrupt: 

#### 7. Using the SQL to filter the category (label)
#### 

In [20]:
import pyspark.sql as sql
count_df = unioned_df.groupBy("category").count()
count_df.createOrReplaceTempView("count_df")
count_rank_df = spark.sql("SELECT category, count FROM count_df ORDER BY count DESC LIMIT 15")

In [21]:
count_rank_df.show(15)

KeyboardInterrupt: 

In [22]:
unioned_df.createOrReplaceTempView("unioned_df")
count_rank_df.createOrReplaceTempView("count_rank_df")
df_final = spark.sql("SELECT * FROM unioned_df WHERE unioned_df.category IN (SELECT category FROM count_rank_df)")

In [ ]:
df_final.show(10)

In [23]:
df_final.agg(countDistinct(df_final.category).alias('distinct_label')).collect()

Py4JJavaError: An error occurred while calling o224.collectToPython.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:150)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:156)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql

In [ ]:
#### Verfied the final data frame 
df_final.groupBy("category").count().show()

In [26]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer, 
                                LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline

In [27]:
import nltk
nltk.download('stopwords')
nltk.download('words')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')
eng_stopwords.append('xxxx')

[nltk_data] Downloading package stopwords to /home/hadoop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/hadoop/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol('context') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]

In [ ]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

In [ ]:
df_pip = pipeline.fit(df_final).transform(df_final)

In [ ]:
df_pip.show()

In [ ]:
from pyspark.sql.functions import explode, col

context_words = df_pip.withColumn("exploded_text", explode(col("finished_clean_lemma")))

In [ ]:
df_pip.select('finished_clean_lemma').show(20, False)

In [ ]:
df_pip.select('np1').show(20,False)

In [ ]:
cols = ['np1', 'finished_clean_lemma', 'category']    

In [ ]:
df_1_new = df_pip.withColumn("np1",col('np1')).select(cols)
df_2_new = df_pip.withColumn("finished_clean_lemma", col('finished_clean_lemma')).select(cols)
result = df_1_new.union(df_2_new)

In [ ]:
result.show()

In [ ]:
,'science',
 'health',
 'technolog',
 'business',
 'sports'

In [12]:
sc.stop()

In [13]:
SparkSession._instantiatedContext = None

In [ ]:

    # process narratives into counts dictionary
    clean_data = pipeline.fit(data).transform(data)
    clean_data_exploded = clean_data.withColumn("exploded_text", explode(col("finished_clean_lemma")))
    counts = clean_data_exploded.groupby('exploded_text').count().toPandas()
    counts_dict = {counts.loc[i, 'exploded_text']: counts.loc[i, 'count'] for i in range(counts.shape[0])}
    
    # add counts to dictionary
    category_word_counts_dict[category] = counts_dict